In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nitime

In [39]:
ts1 = np.loadtxt("ts1.txt")
ts2 = np.loadtxt("ts2.txt")
ts3 = np.loadtxt("ts3.txt")

In [3]:
import numpy as np
from Operations.CO_FirstCrossing import CO_FirstCrossing

def CO_glscf(y, alpha, beta, tau = 'tau'):
    """
    """
    # Set tau to first zero-crossing of the autocorrelation function with the input 'tau'
    if tau == 'tau':
        tau = CO_FirstCrossing(y, 'ac', 0, 'discrete')
    
    # Take magnitudes of time-delayed versions of the time series
    y1 = np.abs(y[:-tau])
    y2 = np.abs(y[tau:])


    p1 = np.mean(np.multiply((y1 ** alpha), (y2 ** beta)))
    p2 = np.multiply(np.mean(y1 ** alpha), np.mean(y2 ** beta))
    p3 = np.sqrt(np.mean(y1 ** (2*alpha)) - (np.mean(y1 ** alpha))**2)
    p4 = np.sqrt(np.mean(y2 ** (2*beta)) - (np.mean(y2 ** beta))**2)

    glscf = (p1 - p2) / (p3 * p4)

    return glscf    


In [11]:
CO_glscf(ts1, 0.9, 0.4, 3)

0.25284356429315435

In [13]:
import numpy as np
from Operations.CO_FirstCrossing import CO_FirstCrossing
from Operations.CO_glscf import CO_glscf

def CO_fzcglscf(y, alpha, beta, maxtau = None):
    """
    """
    N = len(y) # the length of the time series

    if maxtau is None:
        maxtau = N
    
    glscfs = np.zeros(maxtau)

    for i in range(1, maxtau+1):
        tau = i

        glscfs[i-1] = CO_glscf(y, alpha, beta, tau)
        if (i > 1) and (glscfs[i-1]*glscfs[i-2] < 0):
            # Draw a straight line between these two and look at where it hits zero
            out = i - 1 + glscfs[i-1]/(glscfs[i-1]-glscfs[i-2])
            return out
    
    return maxtau # if the function hasn't exited yet, set output to maxtau 


In [86]:
import warnings
import numpy as np

def DN_cv(x, k = 1):
    """
    Coefficient of variation

    Coefficient of variation of order k is sigma^k / mu^k (for sigma, standard
    deviation and mu, mean) of a data vector, x

    Parameters:
    ----------
    x (array-like): The input data vector
    k (int, optional): The order of coefficient of variation (k = 1 is default)

    Returns:
    -------
    float: The coefficient of variation of order k
    """
    if not isinstance(k, int) or k < 0:
        warnings.warn('k should probably be a positive integer')
        # Carry on with just this warning, though
    
    # Compute the coefficient of variation (of order k) of the data
    return (np.std(x, ddof=1) ** k) / (np.mean(x) ** k)


In [88]:
DN_cv(ts3)

26.269342687595913

In [113]:
DN_nlogL_norm(ts2)

1.2765413621752804

In [118]:
import numpy as np 

def DN_HighLowMu(y):
    """
    The highlowmu statistic.

    The highlowmu statistic is the ratio of the mean of the data that is above the
    (global) mean compared to the mean of the data that is below the global mean.

    Paramters:
    ----------
    y (array-like): The input data vector

    Returns:
    --------
    float: The highlowmu statistic.
    """
    mu = np.mean(y) # mean of data
    mhi = np.mean(y[y > mu]) # mean of data above the mean
    mlo = np.mean(y[y < mu]) # mean of data below the mean
    out = np.divide((mhi-mu), (mu-mlo)) # ratio of the differences

    return out


In [119]:
DN_HighLowMu(ts2)

1.0325203252032518

In [120]:
import numpy as np

def BF_SimpleBinner(xData, numBins):
    """
    Generate a histogram from equally spaced bins.
   
    Parameters:
    xData (array-like): A data vector.
    numBins (int): The number of bins.

    Returns:
    tuple: (N, binEdges)
        N (numpy.ndarray): The counts
        binEdges (numpy.ndarray): The extremities of the bins.
    """
    minX = np.min(xData)
    maxX = np.max(xData)
    
    # Linearly spaced bins:
    binEdges = np.linspace(minX, maxX, numBins + 1)
    N = np.zeros(numBins, dtype=int)
    
    for i in range(numBins):
        if i < numBins - 1:
            N[i] = np.sum((xData >= binEdges[i]) & (xData < binEdges[i+1]))
        else:
            # the final bin
            N[i] = np.sum((xData >= binEdges[i]) & (xData <= binEdges[i+1]))
    
    return N, binEdges


In [140]:
from scipy import stats

def DN_Cumulants(y, cumWhatMay = 'skew1'):
    
    """
    """

    if cumWhatMay == 'skew1':
        out = stats.skew(y)
    elif cumWhatMay == 'skew2':
        out = stats.skew(y, bias=False)
    elif cumWhatMay == 'kurt1':
        out = stats.kurtosis(y, fisher=False)
    elif cumWhatMay == 'kurt2':
        out = stats.kurtosis(y, bias=False, fisher=False)
    else:
        raise ValueError('Requested Unknown cumulant must be: skew1, skew2, kurt1, or kurt2')
    
    return out


In [141]:
DN_Cumulants(ts1, cumWhatMay='kurt1')

1.497362974297833

In [142]:
DN_Cumulants(ts1, cumWhatMay='kurt2')

1.4958467355321061

In [144]:
import numpy as np

def EN_ApEN(y, mnom = 1, rth = 0.2):
    """
    """
    r = rth * np.std(y, ddof=1) # threshold of similarity
    N = len(y) # time series length
    phi = np.zeros(2) # phi[0] = phi_m, phi[1] = phi_{m+1}

    for k in range(2):
        m = mnom+k # pattern length
        C = np.zeros(N - m + 1)
        # define the matrix x, containing subsequences of u
        x = np.zeros((N-m+1, m))

        # Form vector sequences x from the time series y
        x = np.array([y[i:i+m] for i in range(N - m + 1)])
        
        for i in range(N - m + 1):
            # Calculate the number of x[j] within r of x[i]
            d = np.abs(x - x[i])
            if m > 1:
                d = np.max(d, axis=1)
            C[i] = np.sum(d <= r) / (N - m + 1)

        phi[k] = np.mean(np.log(C))

    return phi[0] - phi[1]


In [149]:
EN_ApEN(ts3, 2, 0.4)

1.514020217676181

In [152]:
import matplotlib.mlab
import numpy as np
import scipy
from scipy import signal
import math
import matplotlib

def MD_hrv_classic(y):
    """
    """

    # Standard defaults
    y = np.array(y)
    diffy = np.diff(y)
    N = len(y)

    # ------------------------------------------------------------------------------
    # Calculate pNNx percentage
    # ------------------------------------------------------------------------------
    # pNNx: recommendation as per Mietus et. al. 2002, "The pNNx files: ...", Heart
    # strange to do this for a z-scored time series...
    Dy = np.abs(diffy)
    PNNxfn = lambda x : np.mean(Dy > x/1000)

    out = {}

    out['pnn5'] = PNNxfn(5) # 0.0055*sigma
    out['pnn10'] = PNNxfn(10) # 0.01*sigma
    out['pnn20'] = PNNxfn(20) # 0.02*sigma
    out['pnn30'] = PNNxfn(30) # 0.03*sigma
    out['pnn40'] = PNNxfn(40) # 0.04*sigma

    # ------------------------------------------------------------------------------
    # Calculate PSD
    # ------------------------------------------------------------------------------

    F, Pxx = signal.periodogram(y, window=np.hanning(N))

    # Calculate spectral measures such as subband spectral power percentage, LF/HF ratio etc.
    LF_lo = 0.04 # /pi -- fraction of total power (max F is pi)
    LF_hi = 0.15
    HF_lo = 0.15
    HF_hi = 0.4

    fbinsize = F[1] - F[0]

    # Calculate PSD
    f, Pxx = signal.periodogram(y, window='hann', detrend=False)
    f *= 2 * np.pi
    #print(Pxx)

    # Calculate spectral measures
    LF_lo, LF_hi = 0.04, 0.15
    HF_lo, HF_hi = 0.15, 0.4

    fbinsize = f[1] - f[0]
    indl = (f >= LF_lo) & (f <= LF_hi)
    indh = (f >= HF_lo) & (f <= HF_hi)
    indv = f <= LF_lo

    lfp = fbinsize * np.sum(Pxx[indl])
    hfp = fbinsize * np.sum(Pxx[indh])
    vlfp = fbinsize * np.sum(Pxx[indv])
    total = fbinsize * np.sum(Pxx)

    out['lfhf'] = lfp / hfp
    out['vlf'] = vlfp / total * 100
    out['lf'] = lfp / total * 100
    out['hf'] = hfp / total * 100

    # Triangular histogram index
    numBins = 10
    hist = np.histogram(y, bins=numBins)
    out['tri'] = len(y)/np.max(hist[0])

    # Poincare plot measures:
    # cf. "Do Existing Measures ... ", Brennan et. al. (2001), IEEE Trans Biomed Eng 48(11)
    rmssd = np.std(diffy, ddof=1)
    sigma = np.std(y, ddof=1)

    out["SD1"] = 1/math.sqrt(2) * rmssd * 1000
    out["SD2"] = math.sqrt(2 * sigma**2 - (1/2) * rmssd**2) * 1000

    return out


In [242]:
MD_hrv_classic(ts1)

{'pnn5': 0.982982982982983,
 'pnn10': 0.9669669669669669,
 'pnn20': 0.9359359359359359,
 'pnn30': 0.9029029029029029,
 'pnn40': 0.8708708708708709,
 'lfhf': 8.4832685576491e-08,
 'vlf': 1.1046723335508328e-08,
 'lf': 8.483267835924858e-06,
 'hf': 99.99999149238012,
 'tri': 4.830917874396135,
 'SD1': 99.72437694213063,
 'SD2': 996.9513129028788}

In [247]:
import numpy as np


def BF_MakeBuffer(y, bufferSize):
    """
    Make a buffered version of a time series.

    Parameters
    ----------
    y : array-like
        The input time series.
    buffer_size : int
        The length of each buffer segment.

    Returns
    -------
    y_buffer : ndarray
        2D array where each row is a segment of length `buffer_size` 
        corresponding to consecutive, non-overlapping segments of the input time series.
    """
    N = len(y)

    numBuffers = int(np.floor(N/bufferSize))

    # may need trimming
    y_buffer = y[:numBuffers*bufferSize]
    # then reshape
    y_buffer = y_buffer.reshape((numBuffers,bufferSize))

    return y_buffer


In [20]:
import numpy as np

def EN_wentropy(y, whaten = 'shannon', p = None):
    """
    Entropy of time series using wavelets.
    Uses a python port of the MATLAB wavelet toolbox wentropy function.

    Parameters:
    ----------
    y : array_like
        Input time series
    whaten : str, optional
        The entropy type:
        - 'shannon' (default)
        - 'logenergy'
        - 'threshold' (with a given threshold)
        - 'sure' (with a given parameter)
        (see the wentropy documentation for information)
    p : any, optional
        the additional parameter needed for threshold and sure entropies

    Returns:
    --------
    out : float
        Entropy value. 
    """
    N = len(y)

    if whaten == 'shannon':
        # compute Shannon entropy
        out = wentropy(y, 'shannon')/N
    elif whaten == 'logenergy':
        out = wentropy(y, 'logenergy')/N
    elif whaten == 'threshold':
        # check that p has been provided
        if p is not None:
            out = wentropy(y, 'threshold', p)/N
        else:
            raise ValueError("threshold requires an additional parameter, p.")
    elif whaten == 'sure':
        if p is not None:
            out = wentropy(y, 'sure', p)/N
        else:
            raise ValueError("sure requires an additional parameter, p.")
    else:
        raise ValueError(f"Unknown entropy type {whaten}")

    return out

# helper functions
# taken from https://github.com/fairscape/hctsa-py/blob/master/PeripheryFunctions/wentropy.py
def wentropy(x, entType = 'shannon', additionalParameter = None):

    if entType == 'shannon':
        x = np.power(x[ x != 0 ],2)
        return - np.sum(np.multiply(x,np.log(x)))

    elif entType == 'threshold':
        if additionalParameter is None or isinstance(additionalParameter, str):
            return None
        x = np.absolute(x)
        return np.sum((x > additionalParameter))

    elif entType == 'norm':
        if additionalParameter is None or isinstance(additionalParameter,str) or additionalParameter < 1:
            return None
        x = np.absolute(x)
        return np.sum(np.power(x, additionalParameter))

    elif entType == 'sure':
        if additionalParameter is None or isinstance(additionalParameter,str):
            return None

        N = len(x)
        x2 = np.square(x)
        t2 = additionalParameter**2
        xgt = np.sum((x2 > t2))
        xlt = N - xgt

        return N - (2*xlt) + (t2 *xgt) + np.sum(np.multiply(x2,(x2 <= t2)))

    elif entType == 'logenergy':
        x = np.square(x[x != 0])
        return np.sum(np.log(x))

    else:
        print("invalid entropy type")
        return None
    

In [22]:
EN_wentropy(ts2)

-0.22589264096854644

In [23]:

def DN_CustomSkewness(y, whatSkew = 'pearson'):
    """
    Custom skewness measures
    """

    if whatSkew == 'pearson':
        out = ((3 * np.mean(y) - np.median(y)) / np.std(y, ddof=1))
    elif whatSkew == 'bowley':
        qs = np.quantile(y, [0.25, 0.5, 0.75], method='hazen')
        out = (qs[2]+qs[0] - 2 * qs[1]) / (qs[2] - qs[0]) 
    
    return out


In [27]:
DN_CustomSkewness(ts3, 'bowley')

-0.008252488684308562

In [29]:
from scipy import stats
from scipy.stats import norm, genextreme, uniform, beta, rayleigh, expon, gamma, lognorm, weibull_min

In [60]:
def test(x):
    x = (x - np.min(x) + 0.01*np.std(x, ddof=1)) / (np.max(x) - np.min(x) + 0.01*np.std(x, ddof=1))
    params = stats.beta.fit(x)
    print(params)

In [61]:
test(ts1)

(0.5256618923359788, 0.5346381917755811, -0.0930931392643515, 1.0930931392643517)


In [49]:
-2.2148 + 4.6579999999999995

2.4431999999999996

In [62]:
def SC_DFA(y):

    N = len(y)

    tau = int(np.floor(N/2))

    y = y - np.mean(y)

    x = np.cumsum(y)

    taus = np.arange(5,tau+1)

    ntau = len(taus)

    F = np.zeros(ntau)

    for i in range(ntau):

        t = int(taus[i])



        x_buff = x[:N - N % t]

        x_buff = x_buff.reshape((int(N / t),t))


        y_buff = np.zeros((int(N / t),t))

        for j in range(int(N / t)):

            tt = range(0,int(t))

            p = np.polyfit(tt,x_buff[j,:],1)

            y_buff[j,:] =  np.power(x_buff[j,:] - np.polyval(p,tt),2)



        y_buff.reshape((N - N % t,1))

        F[i] = np.sqrt(np.mean(y_buff))

    logtaur = np.log(taus)

    logF = np.log(F)

    p = np.polyfit(logtaur,logF,1)

    return p[0]

In [64]:
SC_DFA(ts2)

0.2856214933647925

In [65]:
import numpy as np
import math 

def CO_RM_AMInformation(y,tau = 1):
    """
    A wrapper for rm_information(), which calculates automutal information

    Inputs:
        y, the input time series
        tau, the time lag at which to calculate automutal information

    :returns estimate of mutual information

    - Wrapper initially developed by Ben D. Fulcher in MATLAB
    - rm_information.py initially developed by Rudy Moddemeijer in MATLAB
    - Translated to python by Tucker Cullen

    """

    if tau >= len(y):

        return

    y1 = y[:-tau]
    y2 = y[tau:]

    out = RM_information(y1,y2)

    return out[0]

def RM_histogram2(*args):
    """
    rm_histogram2() computes the two dimensional frequency histogram of two row vectors x and y

    Takes in either two or three parameters:
        rm_histogram(x, y)
        rm_histogram(x, y, descriptor)

    x, y : the row vectors to be analyzed
    descriptor : the descriptor of the histogram where:

        descriptor = [lowerx, upperx, ncellx, lowery, uppery, ncelly]
            lower? : the lowerbound of the ? dimension of the histogram
            upper? : the upperbound of the dimension of the histogram
            ncell? : the number of cells of the ? dimension of the histogram

    :return: a tuple countaining a) the result (the 2d frequency histogram), b) descriptor (the descriptor used)

    MATLAB function and logic by Rudy Moddemeijer
    Translated to python by Tucker Cullen

    """

    nargin = len(args)

    if nargin < 1:
        print("Usage: result = rm_histogram2(X, Y)")
        print("       result = rm_histogram2(X,Y)")
        print("Where: descriptor = [lowerX, upperX, ncellX; lowerY, upperY, ncellY")

    # some initial tests on the input arguments

    x = np.array(args[0])  # make sure the imputs are in numpy array form
    y = np.array(args[1])

    xshape = x.shape
    yshape = y.shape

    lenx = xshape[0]  # how many elements are in the row vector
    leny = yshape[0]

    if len(xshape) != 1:  # makes sure x is a row vector
        print("Error: invalid dimension of x")
        return

    if len(yshape) != 1:
        print("Error: invalid dimension of y")
        return

    if lenx != leny:  # makes sure x and y have the same amount of elements
        print("Error: unequal length of x and y")
        return

    if nargin > 3:
        print("Error: too many arguments")
        return

    if nargin == 2:
        minx = np.amin(x)
        maxx = np.amax(x)
        deltax = (maxx - minx) / (lenx - 1)
        ncellx = math.ceil(lenx ** (1 / 3))

        miny = np.amin(y)
        maxy = np.amax(y)
        deltay = (maxy - miny) / (leny - 1)
        ncelly = ncellx
        descriptor = np.array(
            [[minx - deltax / 2, maxx + deltax / 2, ncellx], [miny - deltay / 2, maxy + deltay / 2, ncelly]])
    else:
        descriptor = args[2]

    lowerx = descriptor[0, 0]  # python indexes one less then matlab indexes, since starts at zero
    upperx = descriptor[0, 1]
    ncellx = descriptor[0, 2]
    lowery = descriptor[1, 0]
    uppery = descriptor[1, 1]
    ncelly = descriptor[1, 2]

    # checking descriptor to make sure it is valid, otherwise print an error

    if ncellx < 1:
        print("Error: invalid number of cells in X dimension")

    if ncelly < 1:
        print("Error: invalid number of cells in Y dimension")

    if upperx <= lowerx:
        print("Error: invalid bounds in X dimension")

    if uppery <= lowery:
        print("Error: invalid bounds in Y dimension")

    result = np.zeros([int(ncellx), int(ncelly)],
                      dtype=int)  # should do the same thing as matlab: result(1:ncellx,1:ncelly) = 0;

    xx = np.around((x - lowerx) / (upperx - lowerx) * ncellx + 1 / 2)
    yy = np.around((y - lowery) / (uppery - lowery) * ncelly + 1 / 2)

    xx = xx.astype(int)  # cast all the values in xx and yy to ints for use in indexing, already rounded in previous step
    yy = yy.astype(int)

    for n in range(0, lenx):
        indexx = xx[n]
        indexy = yy[n]

        indexx -= 1  # adjust indices to start at zero, not one like in MATLAB
        indexy -= 1

        if indexx >= 0 and indexx <= ncellx - 1 and indexy >= 0 and indexy <= ncelly - 1:
            result[indexx, indexy] = result[indexx, indexy] + 1

    return result, descriptor

def RM_information(*args):
    """
    rm_information estimates the mutual information of the two stationary signals with
    independent pairs of samples using various approaches:

    takes in between 2 and 5 parameters:
        rm_information(x, y)
        rm_information(x, y, descriptor)
        rm_information(x, y, descriptor, approach)
        rm_information(x, y, descriptor, approach, base)

    :returns estimate, nbias, sigma, descriptor

        estimate : the mututal information estimate
        nbias : n-bias of the estimate
        sigma : the standard error of the estimate
        descriptor : the descriptor of the histogram, see also rm_histogram2

            lowerbound? : lowerbound of the histogram in the ? direction
            upperbound? : upperbound of the histogram in the ? direction
            ncell? : number of cells in the histogram in ? direction

        approach : method used, choose from the following:

            'unbiased'  : the unbiased estimate (default)
            'mmse'      : minimum mean square estimate
            'biased'    : the biased estimate

        base : the base of the logarithm, default e

    MATLAB function and logic by Rudy Moddemeijer
    Translated to python by Tucker Cullen
    """

    nargin = len(args)

    if nargin < 1:
        print("Takes in 2-5 parameters: ")
        print("rm_information(x, y)")
        print("rm_information(x, y, descriptor)")
        print("rm_information(x, y, descriptor, approach)")
        print("rm_information(x, y, descriptor, approach, base)")
        print()

        print("Returns a tuple containing: ")
        print("estimate, nbias, sigma, descriptor")
        return

    # some initial tests on the input arguments

    x = np.array(args[0])  # make sure the imputs are in numpy array form
    y = np.array(args[1])

    xshape = x.shape
    yshape = y.shape

    lenx = xshape[0]  # how many elements are in the row vector
    leny = yshape[0]

    if len(xshape) != 1:  # makes sure x is a row vector
        print("Error: invalid dimension of x")
        return

    if len(yshape) != 1:
        print("Error: invalid dimension of y")
        return

    if lenx != leny:  # makes sure x and y have the same amount of elements
        print("Error: unequal length of x and y")
        return

    if nargin > 5:
        print("Error: too many arguments")
        return

    if nargin < 2:
        print("Error: not enough arguments")
        return

    # setting up variables depending on amount of inputs

    if nargin == 2:
        hist = RM_histogram2(x, y)  # call outside function from rm_histogram2.py
        h = hist[0]
        descriptor = hist[1]

    if nargin >= 3:
        hist = RM_histogram2(x, y, args[2])  # call outside function from rm_histogram2.py, args[2] represents the given descriptor
        h = hist[0]
        descriptor = hist[1]

    if nargin < 4:
        approach = 'unbiased'
    else:
        approach = args[3]

    if nargin < 5:
        base = math.e  # as in e = 2.71828
    else:
        base = args[4]

    lowerboundx = descriptor[0, 0]  #not sure why most of these were included in the matlab script, most of them go unused
    upperboundx = descriptor[0, 1]
    ncellx = descriptor[0, 2]
    lowerboundy = descriptor[1, 0]
    upperboundy = descriptor[1, 1]
    ncelly = descriptor[1, 2]

    estimate = 0
    sigma = 0
    count = 0

    # determine row and column sums

    hy = np.sum(h, 0)
    hx = np.sum(h, 1)

    ncellx = ncellx.astype(int)
    ncelly = ncelly.astype(int)

    for nx in range(0, ncellx):
        for ny in range(0, ncelly):
            if h[nx, ny] != 0:
                logf = math.log(h[nx, ny] / hx[nx] / hy[ny])
            else:
                logf = 0

            count = count + h[nx, ny]
            estimate = estimate + h[nx, ny] * logf
            sigma = sigma + h[nx, ny] * (logf ** 2)

    # biased estimate

    estimate = estimate / count
    sigma = math.sqrt((sigma / count - estimate ** 2) / (count - 1))
    estimate = estimate + math.log(count)
    nbias = (ncellx - 1) * (ncelly - 1) / (2 * count)

    # conversion to unbiased estimate

    if approach[0] == 'u':
        estimate = estimate - nbias
        nbias = 0

        # conversion to minimum mse estimate

    if approach[0] == 'm':
        estimate = estimate - nbias
        nbias = 0
        lamda = (estimate ** 2) / ((estimate ** 2) + (sigma ** 2))
        nbias = (1 - lamda) * estimate
        estimate = lamda * estimate
        sigma = lamda * sigma

        # base transformations

    estimate = estimate / math.log(base)
    nbias = nbias / math.log(base)
    sigma = sigma / math.log(base)

    return estimate, nbias, sigma, descriptor


In [70]:
CO_RM_AMInformation(ts3, 15)

-0.010575122175197689